### Import 및 전처리


In [ ]:
from ast import literal_eval

import pandas as pd
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_pinecone import PineconeVectorStore

from rag_modules import *
from utils import get_pinecone_index

In [ ]:
def read_csv(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)

    df["question_plus"] = df["question_plus"].fillna(" ")
    df["choices"] = df["choices"].apply(literal_eval)
    df["choices_text"] = df["choices"].apply(lambda x: " ".join(x))

    if "keyword_1" in df.columns:
        df["keywords"] = df[["keyword_1", "keyword_2", "keyword_3", "keyword_4", "keyword_5"]].apply(
            lambda row: row.tolist(), axis=1
        )

    if "keyword_1_exists" in df.columns:
        df["keywords_exists"] = df[
            ["keyword_1_exists", "keyword_2_exists", "keyword_3_exists", "keyword_4_exists", "keyword_5_exists"]
        ].apply(lambda row: row.astype(int).tolist(), axis=1)

    return df

### 실험이 가능한 함수 리스트

- tf_idf_query_builder
- original_paragraph_query_builder
- original_question_query_builder
- original_choices_query_builder
- original_question_plus_query_builder
- original_summarization_query_builder
- original_keywords_query_builder
- original_exist_keywords_query_builder


In [ ]:
def rag_retrieval(row: pd.Series):
    pinecone_index, index_name = get_pinecone_index()
    embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-m3")
    vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings, text_key="text")

    # 수정이 필요한 부분
    query_builder = original_exist_keywords_query_builder()

    query = query_builder.build(row)
    print("Builded Query:")
    print(query)
    retrieved_docs = vector_store.similarity_search(query, k=5)  # k는 반환할 문서 개수
    # 검색된 문서 출력
    print("Retrieved Documents:")
    for i, doc in enumerate(retrieved_docs):
        print(f"{i + 1}. {doc.page_content}")
        print(f"   Metadata: {doc.metadata}")
    print()


if __name__ == "__main__":
    df = read_csv("data/train_with_summarization.csv")
    row = df.iloc[1]
    rag_retrieval(row)

Connected to Pinecone index: gen-nlp-bge-m3
Builded Query:
삼한 수덕 고려사 봉기
Retrieved Documents:
1. (중국어) 세친 조, 현장 한역 (T.1558). 《아비달마구사론(阿毘達磨俱舍論)》. 대정신수대장경. T29, No. 1558, CBETA.  
(중국어) 호법 등 지음, 현장 한역 (T.1585). 《성유식론(成唯識論)》. 대정신수대장경. T31, No. 1585, CBETA.
   Metadata: {'chunk_index': 6.0, 'document_id': '내외', 'page_index': 415.0}
2. (중국어) 세친 조, 현장 한역 (T.1558). 《아비달마구사론(阿毘達磨俱舍論)》. 대정신수대장경. T29, No. 1558, CBETA.  
(중국어) 호법 등 지음, 현장 한역 (T.1585). 《성유식론(成唯識論)》. 대정신수대장경. T31, No. 1585, CBETA.
   Metadata: {'chunk_index': 6.0, 'document_id': '내외', 'page_index': 416.0}
3. 같이 보기
폭행위협
대한민국 형법 제260조

참고 문헌
서철원,《미국 불법행위법》,법원사, 2005. (ISBN 89-91512-01-1)
이상윤,《영미법》,박영사, 2003.
Black's Law Dictionary, 1486 (8th ed. 2004).
   Metadata: {'chunk_index': 8.0, 'document_id': '폭행죄', 'page_index': 369.0}
4. 서적
탁양현 (2020). 《자유민주주의 사회민주주의 인민민주주의: 대한민국은 자유민주주의 국가다》. 키메이커. 36쪽. ISBN 979-11-6569-118-9  – e퍼플 경유.  
이희진 (2004년 9월 1일). 〈4. 백제의 패배인가? | 관산성 전투〉. 《전쟁의 발견 -한국 고대사의 재구성을 위하여-》. 동아시아. ISBN 9788988165461. 
KBS